In [1]:
#Load necessary packages
import pandas as pd
import numpy as np
import xgboost as xgb
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from scipy.sparse import coo_matrix, hstack
import scipy as sp
from scipy.sparse import csr_matrix
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from matplotlib import pyplot
#import sys  
import string
import collections
from sklearn import svm

# reload(sys)  
# sys.setdefaultencoding('utf8')

C:\Users\Admin\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)

In [73]:
train = pd.read_csv(r"C:\Users\Admin\Desktop\Kaggle\Toxic Comment Classification\train.csv")
test = pd.read_csv (r"C:\Users\Admin\Desktop\Kaggle\Toxic Comment Classification\test.csv")
submission = pd.read_csv(r"C:\Users\Admin\Desktop\Kaggle\Toxic Comment Classification\sample_submission.csv")

In [5]:
# #Crazy Stacking . Load the submission file with filled probabilities
# submission = pd.read_csv(r"C:\Users\Admin\Desktop\submission.csv")

In [6]:
# #new train is test+train
# train2 = pd.merge(test,submission)

In [7]:
# #Crazy Stacking
# train = train.append(train2)

In [10]:
# #Let's get the ratio of 1's to 0's
# train["toxic"].value_counts()

In [11]:
# So the class is imalanced , the highest being toxic

# CLEANING AND COMBINING#

In [74]:
#let's fill the na's in the data
train = train.fillna(' ')#Change made
test = test.fillna(' ')#Change made

In [75]:
#Combining text to create features
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [6]:
# #Cleaning and adding stop words
# #stops = set(stopwords.words("english"))
# import re
# clean_rows = []
# for rows in all_text:
#     rows = rows.lower()
#     rows = re.sub("&lt;3"," good ",rows)
#     rows = re.sub(",s", " sad ",rows)
#     rows = re.sub(",-s", " sad ",rows)
#     rows = re.sub(r"\br\b", "are",rows)
#     rows = re.sub(r"\bu\b", "you",rows)
#     rows = re.sub(r"\bhaha\b", "ha",rows)
#     rows = re.sub(r"\bhahaha\b", "ha",rows)
#     rows = re.sub(r"\bdon't\b", "do not",rows)
#     rows = re.sub(r"\bdoesn't\b", "does not",rows)
#     rows = re.sub(r"\bdidn't\b", "did not",rows)
#     rows = re.sub(r"\bhasn't\b", "has not",rows)
#     rows = re.sub(r"\bhaven't\b", "have not",rows)
#     rows = re.sub(r"\bhadn't\b", "had not",rows)
#     rows = re.sub(r"\bwon't\b", "will not",rows)
#     rows = re.sub(r"\bwouldn't\b", "would not",rows)
#     rows= re.sub(r"\bcan't\b", "can not",rows)
#     rows = re.sub(r"\bcannot\b", "can not",rows)
#     rows = re.sub(r"\bi'm\b", "i am",rows)
#     rows = re.sub("m", "am",rows)
#     rows = re.sub("r", "are",rows)
#     rows = re.sub("u", "you",rows)
#     rows = re.sub("haha", "ha",rows)
#     rows = re.sub("hahaha", "ha",rows)
#     rows = re.sub("don't", "do not",rows)
#     rows = re.sub("doesn't", "does not",rows)
#     rows = re.sub("didn't", "did not",rows)
#     rows = re.sub("hasn't", "has not",rows)
#     rows = re.sub("haven't", "have not",rows)
#     rows = re.sub("hadn't", "had not",rows)
#     rows = re.sub("won't", "will not",rows)
#     rows = re.sub("wouldn't", "would not",rows)
#     rows = re.sub("can't", "can not",rows)
#     rows = re.sub("cannot", "can not",rows)
#     rows = re.sub("i'm", "i am",rows)
#     rows = re.sub("m", "am",rows)
#     rows = re.sub("i'll" , "i will",rows)
#     rows = re.sub("its" , "it is",rows)
#     rows = re.sub("it's" , "it is",rows)
#     rows = re.sub("'s" , " is",rows)
#     rows = re.sub("that's" , "that is",rows)
#     rows = re.sub("weren't" , "were not",rows)
#     #rows = re.sub('[^A-Za-z]+', ' ', rows)
#     clean_rows.append(rows)
# all_text = clean_rows

In [215]:
# #Bag Of words
# count_vectorizer = CountVectorizer(
#     analyzer="word", tokenizer=nltk.word_tokenize,
#     preprocessor=None, stop_words='english', max_features=10000)  
# bag_of_words = count_vectorizer.fit(all_text)
# train_bow_features = count_vectorizer.transform(train_text)
# test_bow_features = count_vectorizer.transform(test_text)

In [76]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    min_df=3,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 3),
    use_idf=1,
    smooth_idf=1,
    max_features=20000)

In [77]:
word_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=3,
        ngram_range=(1, 3), norm=u'l2', preprocessor=None, smooth_idf=1,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=1,
        vocabulary=None)

In [78]:
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [23]:
# svd = TruncatedSVD(n_components=199)
# train_word_features = svd.fit_transform(train_word_features)
# test_word_features = svd.fit_transform(test_word_features)

In [79]:
count_vectorizer = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
                                   ngram_range=(1, 3)#change made
                                   , stop_words = 'english',max_features = 20000)

In [80]:
count_vectorizer.fit(all_text)

CountVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\w{1,}', tokenizer=None,
        vocabulary=None)

In [81]:
train_count_features = count_vectorizer.transform(train_text)
test_count_features = count_vectorizer.transform(test_text)

In [13]:
#glove embedding
# load the GloVe vectors in a dictionary:
embeddings_index = {}
f = open(r'C:\Users\Admin\Desktop\Glove Embedding\glove.840B.300d\glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

2196017it [02:27, 14924.58it/s]


In [14]:
print('Found %s word vectors.' % len(embeddings_index))

Found 2196016 word vectors.


In [15]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower().decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [16]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in train_text]
xvalid_glove = [sent2vec(x) for x in test_text]

In [17]:
xtrain_glove = np.array(xtrain_glove)
xtest_glove = np.array(xvalid_glove)

In [229]:
# char_vectorizer = TfidfVectorizer(
#     sublinear_tf=True,
#     strip_accents='unicode',
#     analyzer='char',
#     stop_words='english',
#     ngram_range=(1, 3),
#     max_features=20000)
# char_vectorizer.fit(all_text)
# train_char_features = char_vectorizer.transform(train_text)
# test_char_features = char_vectorizer.transform(test_text)

In [34]:
# svd = TruncatedSVD(n_components=25, n_iter=25, random_state=12)
# train_char_features = svd.fit_transform(train_char_features)
# test_char_features = svd.fit_transform(test_char_features)

In [35]:
# svd = TruncatedSVD(n_components=25, n_iter=25, random_state=12)
# train_char_features = svd.fit_transform(train_bow_features)
# test_char_features = svd.fit_transform(train_bow_features)

In [36]:
# svd = TruncatedSVD(n_components=50)
# train_count_features = svd.fit_transform(train_count_features)
# test_count_features = svd.fit_transform(train_count_features)

In [82]:
train['total_length'] = train['comment_text'].apply(len)
train['capitals'] = train['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
train['caps_vs_length'] = train['caps_vs_length'] = train.apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                axis=1)
train['num_exclamation_marks'] = train['comment_text'].apply(lambda comment: comment.count('!'))
train['num_question_marks'] = train['comment_text'].apply(lambda comment: comment.count('?'))
train['num_punctuation'] = train['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in '.,;:'))
train['num_symbols'] = train['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in '*&$%'))
train['num_words'] = train['comment_text'].apply(lambda comment: len(comment.split()))
train['num_unique_words'] = train['comment_text'].apply(
    lambda comment: len(set(w for w in comment.split())))
train['words_vs_unique'] = train['num_unique_words'] / train['num_words']
train['num_smilies'] = train['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))

In [83]:
test['total_length'] = test['comment_text'].apply(len)
test['capitals'] = test['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
test['caps_vs_length'] = test.apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                axis=1)
test['num_exclamation_marks'] = test['comment_text'].apply(lambda comment: comment.count('!'))
test['num_question_marks'] = test['comment_text'].apply(lambda comment: comment.count('?'))
test['num_punctuation'] = test['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in '.,;:'))
test['num_symbols'] = test['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in '*&$%'))
test['num_words'] = test['comment_text'].apply(lambda comment: len(comment.split()))
test['num_unique_words'] = test['comment_text'].apply(
    lambda comment: len(set(w for w in comment.split())))
test['words_vs_unique'] = test['num_unique_words'] / test['num_words']
test['num_smilies'] = test['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))

In [84]:
# train['total_length'] = train['comment_text'].apply(len)

train['words'] = train['comment_text'].apply(lambda comment: len(comment.split()))
# train['words_vs_length'] = train['words'] / train['total_length']

# train['capitals'] = train['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
# train['capitals_vs_length'] = train['capitals'] / train['total_length']
# train['capitals_vs_words'] = train['capitals'] / train['words']

# train['paragraphs'] = train['comment_text'].apply(lambda comment: comment.count('\n'))
# train['paragraphs_vs_length'] = train['paragraphs'] / train['total_length']
# train['paragraphs_vs_words'] = train['paragraphs'] / train['words']

# eng_stopwords = set(stopwords.words("english"))
# train['stopwords'] = train['comment_text'].apply(lambda comment: sum(comment.count(w) for w in eng_stopwords))
# train['stopwords_vs_length'] = train['stopwords'] / train['total_length']
# train['stopwords_vs_words'] = train['stopwords'] / train['words']

# train['exclamation_marks'] = train['comment_text'].apply(lambda comment: comment.count('!'))
# train['exclamation_marks_vs_length'] = train['exclamation_marks'] / train['total_length']
# train['exclamation_marks_vs_words'] = train['exclamation_marks'] / train['words']

# train['question_marks'] = train['comment_text'].apply(lambda comment: comment.count('?'))
# train['question_marks_vs_length'] = train['question_marks'] / train['total_length']
# train['question_marks_vs_words'] = train['question_marks'] / train['words']

# train['punctuation'] = train['comment_text'].apply(lambda comment: sum(comment.count(w) for w in string.punctuation))
# train['punctuation_vs_length'] = train['punctuation'] / train['total_length']
# train['punctuation_vs_words'] = train['punctuation'] / train['words']

# train['unique_words'] = train['comment_text'].apply(
#     lambda comment: len(set(w for w in comment.split())))
# train['unique_words_vs_length'] = train['unique_words'] / train['total_length']
# train['unique_words_vs_words'] = train['unique_words'] / train['words']

# repeated_threshold = 15
# def count_repeated(text):
#     text_splitted = text.split()
#     word_counts = collections.Counter(text_splitted)
#     return sum(count for word, count in sorted(word_counts.items()) if count > repeated_threshold)

# train['repeated_words'] = train['comment_text'].apply(lambda comment: count_repeated(comment))
# train['repeated_words_vs_length'] = train['repeated_words'] / train['total_length']
# train['repeated_words_vs_words'] = train['repeated_words'] / train['words']

# train['mentions'] = train['comment_text'].apply(
#     lambda comment: comment.count("User:"))
# train['mentions_vs_length'] = train['mentions'] / train['total_length']
# train['mentions_vs_words'] = train['mentions'] / train['words']


train['smilies'] = train['comment_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
# train['smilies_vs_length'] = train['smilies'] / train['total_length']
# train['smilies_vs_words'] = train['smilies'] / train['words']

# train['symbols'] = train['comment_text'].apply(
#     lambda comment: sum(comment.count(w) for w in '*&#$%“”¨«»®´·º½¾¿¡§£₤‘’'))
# train['symbols_vs_length'] = train['symbols'] / train['total_length']
# train['symbols_vs_words'] = train['symbols'] / train['words']

In [85]:
# test['total_length'] = test['comment_text'].apply(len)

test['words'] = test['comment_text'].apply(lambda comment: len(comment.split()))
# test['words_vs_length'] = test['words'] / test['total_length']

# test['capitals'] = test['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
# test['capitals_vs_length'] = test['capitals'] / test['total_length']
# test['capitals_vs_words'] = test['capitals'] / test['words']

# test['paragraphs'] = test['comment_text'].apply(lambda comment: comment.count('\n'))
# test['paragraphs_vs_length'] = test['paragraphs'] / test['total_length']
# test['paragraphs_vs_words'] = test['paragraphs'] / test['words']

# eng_stopwords = set(stopwords.words("english"))
# test['stopwords'] = test['comment_text'].apply(lambda comment: sum(comment.count(w) for w in eng_stopwords))
# test['stopwords_vs_length'] = test['stopwords'] / test['total_length']
# test['stopwords_vs_words'] = test['stopwords'] / test['words']

# test['exclamation_marks'] = test['comment_text'].apply(lambda comment: comment.count('!'))
# test['exclamation_marks_vs_length'] = test['exclamation_marks'] / test['total_length']
# test['exclamation_marks_vs_words'] = test['exclamation_marks'] / test['words']

# test['question_marks'] = test['comment_text'].apply(lambda comment: comment.count('?'))
# test['question_marks_vs_length'] = test['question_marks'] / test['total_length']
# test['question_marks_vs_words'] = test['question_marks'] / test['words']

# test['punctuation'] = test['comment_text'].apply(
#     lambda comment: sum(comment.count(w) for w in string.punctuation))
# test['punctuation_vs_length'] = test['punctuation'] / test['total_length']
# test['punctuation_vs_words'] = test['punctuation'] / test['words']

# test['unique_words'] = test['comment_text'].apply(
#     lambda comment: len(set(w for w in comment.split())))
# test['unique_words_vs_length'] = test['unique_words'] / test['total_length']
# test['unique_words_vs_words'] = test['unique_words'] / test['words']

# repeated_threshold = 15
# def count_repeated(text):
#     text_splitted = text.split()
#     word_counts = collections.Counter(text_splitted)
#     return sum(count for word, count in sorted(word_counts.items()) if count > repeated_threshold)

# test['repeated_words'] = test['comment_text'].apply(lambda comment: count_repeated(comment))
# test['repeated_words_vs_length'] = test['repeated_words'] / test['total_length']
# test['repeated_words_vs_words'] = test['repeated_words'] / test['words']

# test['mentions'] = test['comment_text'].apply(
#     lambda comment: comment.count("User:"))
# test['mentions_vs_length'] = test['mentions'] / test['total_length']
# test['mentions_vs_words'] = test['mentions'] / test['words']


test['smilies'] = test['comment_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
# test['smilies_vs_length'] = test['smilies'] / test['total_length']
# test['smilies_vs_words'] = test['smilies'] / test['words']

# test['symbols'] = test['comment_text'].apply(
#     lambda comment: sum(comment.count(w) for w in '*&#$%“”¨«»®´·º½¾¿¡§£₤‘’'))
# test['symbols_vs_length'] = test['symbols'] / test['total_length']
# test['symbols_vs_words'] = test['symbols'] / test['words']

In [86]:
#Merge all features.
x_train = hstack((
    train_word_features,train_count_features,xtrain_glove,
    csr_matrix(np.reshape(train['caps_vs_length'].values, (train.shape[0], 1))),
    csr_matrix(np.reshape(train['num_unique_words'].values, (train.shape[0], 1))),
    csr_matrix(np.reshape(train['num_exclamation_marks'].values, (train.shape[0], 1))),
    csr_matrix(np.reshape(train['num_punctuation'].values, (train.shape[0], 1))),
    csr_matrix(np.reshape(train['capitals'].values, (train.shape[0], 1))),
    csr_matrix(np.reshape(train['words'].values, (train.shape[0], 1)))
    #csr_matrix(np.reshape(train['smilies'].values, (train.shape[0], 1)))
 ))

In [87]:
x_test = hstack((
    test_word_features,test_count_features,xtest_glove,
    csr_matrix(np.reshape(test['caps_vs_length'].values, (test.shape[0], 1))),
    csr_matrix(np.reshape(test['num_unique_words'].values, (test.shape[0], 1))),
    csr_matrix(np.reshape(test['num_exclamation_marks'].values, (test.shape[0], 1))),
    csr_matrix(np.reshape(test['num_punctuation'].values, (test.shape[0], 1))),
    csr_matrix(np.reshape(test['capitals'].values, (test.shape[0], 1))),
    csr_matrix(np.reshape(test['words'].values, (test.shape[0], 1)))
    #csr_matrix(np.reshape(test['smilies'].values, (test.shape[0], 1)))
    
))

In [ ]:
# #best parameters for XGB
# param['objective'] = 'binary:logistic'

# param['eta'] = 0.095

# param['max_depth'] = 5

# param['silent'] = 1

# param['eval_metric'] = 'auc'

# param['min_child_weight'] = 2

# param['subsample'] = 0.7

# param['colsample_bytree'] = 0.7

In [ ]:
# #Splitting the data into train and test sets using SVD feature
# y_train = train['toxic']
# x_train, x_valid, y_train, y_valid = train_test_split(train_word_features, y_train, test_size=0.10)

# TOXIC#

In [88]:
y_train = train['toxic']
x_train = x_train
#x_train = train.drop(['id','toxic','severe_toxic','insult','threat','obscene','identity_hate','comment_text'],axis=1)
#x_train = xtrain_glove
#x_train = train_word_features
#x_train = train_count_features

In [ ]:
#x_test = xtest_glove

In [ ]:
#x_test = test.drop(['id','comment_text'],axis=1)

In [89]:
xgtrain = xgb.DMatrix(x_train, y_train)

In [90]:
clf = xgb.XGBClassifier(max_depth = 3,
                n_estimators=5000,
                learning_rate=0.1, 
                gamma = 0.5,
                reg_lambda = 0.05,
                reg_alpha = 0.05,
                #nthread=4,
                subsample=0.7,
                colsample_bytree=0.7,
                min_child_weight = 1,
                seed=1301)
xgb_param = clf.get_xgb_params()

In [91]:
#do cross validation
print ('Start cross validation')
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=5000, nfold=5, metrics=['auc'],
     early_stopping_rounds=60, stratified=True, seed=1301,verbose_eval = 1)

Start cross validation
[0]	train-auc:0.784142+0.0399973	test-auc:0.783268+0.0396815
[1]	train-auc:0.844618+0.00725986	test-auc:0.843176+0.00743294
[2]	train-auc:0.856373+0.00987286	test-auc:0.855438+0.00900437
[3]	train-auc:0.866961+0.00849282	test-auc:0.865217+0.00705459
[4]	train-auc:0.877604+0.0100684	test-auc:0.876273+0.00859499
[5]	train-auc:0.893248+0.00281371	test-auc:0.890795+0.00617078
[6]	train-auc:0.900349+0.00204957	test-auc:0.898112+0.00598911
[7]	train-auc:0.907111+0.00145439	test-auc:0.904048+0.00473729
[8]	train-auc:0.91053+0.00225516	test-auc:0.907887+0.00389431
[9]	train-auc:0.914452+0.00186109	test-auc:0.911607+0.00369718
[10]	train-auc:0.91732+0.000917937	test-auc:0.91466+0.00310988
[11]	train-auc:0.919629+0.0010142	test-auc:0.917032+0.00355336
[12]	train-auc:0.922349+0.00133716	test-auc:0.919776+0.00328702
[13]	train-auc:0.924189+0.000678036	test-auc:0.921272+0.00383267
[14]	train-auc:0.92591+0.00113698	test-auc:0.922875+0.00404085
[15]	train-auc:0.92795+0.00122538

[127]	train-auc:0.972194+0.000351934	test-auc:0.966112+0.00162375
[128]	train-auc:0.972289+0.000384024	test-auc:0.966182+0.00161453
[129]	train-auc:0.972393+0.000351119	test-auc:0.966229+0.00161136
[130]	train-auc:0.972496+0.000349249	test-auc:0.966288+0.00160061
[131]	train-auc:0.972598+0.000353006	test-auc:0.966368+0.00161328
[132]	train-auc:0.972733+0.000382509	test-auc:0.966423+0.00162257
[133]	train-auc:0.972801+0.000375245	test-auc:0.96649+0.00163953
[134]	train-auc:0.97291+0.000390734	test-auc:0.966586+0.00163297
[135]	train-auc:0.97301+0.000399867	test-auc:0.966657+0.0016115
[136]	train-auc:0.973106+0.000389214	test-auc:0.966722+0.0016136
[137]	train-auc:0.973171+0.000383328	test-auc:0.966786+0.00160535
[138]	train-auc:0.973255+0.000391806	test-auc:0.966866+0.00161947
[139]	train-auc:0.973345+0.000380154	test-auc:0.966923+0.00159655
[140]	train-auc:0.973443+0.000374844	test-auc:0.966985+0.00158251
[141]	train-auc:0.973523+0.000370897	test-auc:0.967038+0.00158465
[142]	train-auc

[252]	train-auc:0.98024+0.000323506	test-auc:0.971375+0.00144146
[253]	train-auc:0.980282+0.000312787	test-auc:0.971394+0.00143668
[254]	train-auc:0.980322+0.000321672	test-auc:0.97142+0.00143344
[255]	train-auc:0.980361+0.000317525	test-auc:0.971448+0.00143715
[256]	train-auc:0.980408+0.000306644	test-auc:0.97147+0.00144583
[257]	train-auc:0.980439+0.000313217	test-auc:0.971483+0.00144491
[258]	train-auc:0.980494+0.000309474	test-auc:0.971512+0.00143881
[259]	train-auc:0.980543+0.000301831	test-auc:0.971529+0.0014368
[260]	train-auc:0.980579+0.000304188	test-auc:0.971538+0.00143372
[261]	train-auc:0.980613+0.000302681	test-auc:0.971556+0.00143911
[262]	train-auc:0.980655+0.000298532	test-auc:0.971584+0.00142864
[263]	train-auc:0.980702+0.00028572	test-auc:0.971603+0.00142494
[264]	train-auc:0.980746+0.000272818	test-auc:0.97163+0.00142337
[265]	train-auc:0.980782+0.000272919	test-auc:0.971639+0.00142247
[266]	train-auc:0.980829+0.000282795	test-auc:0.971653+0.00141847
[267]	train-auc:

[377]	train-auc:0.984589+0.000245311	test-auc:0.973254+0.0013507
[378]	train-auc:0.98461+0.000241418	test-auc:0.973256+0.00135374
[379]	train-auc:0.984638+0.000242359	test-auc:0.973268+0.00134923
[380]	train-auc:0.984658+0.000240871	test-auc:0.973276+0.00134515
[381]	train-auc:0.984685+0.000239768	test-auc:0.973286+0.00134525
[382]	train-auc:0.984717+0.00023441	test-auc:0.97329+0.00134394
[383]	train-auc:0.98474+0.000242841	test-auc:0.973306+0.00134553
[384]	train-auc:0.98477+0.000234578	test-auc:0.973326+0.00135544
[385]	train-auc:0.984804+0.000235942	test-auc:0.973344+0.00135015
[386]	train-auc:0.984833+0.00024316	test-auc:0.973349+0.00135109
[387]	train-auc:0.984857+0.000243651	test-auc:0.973358+0.0013358
[388]	train-auc:0.984902+0.000231155	test-auc:0.973368+0.00134699
[389]	train-auc:0.98493+0.000236947	test-auc:0.97336+0.00135025
[390]	train-auc:0.984951+0.000233265	test-auc:0.973365+0.00134694
[391]	train-auc:0.984982+0.000237199	test-auc:0.973372+0.00134372
[392]	train-auc:0.98

[502]	train-auc:0.987591+0.000172742	test-auc:0.974186+0.00134452
[503]	train-auc:0.987611+0.000171134	test-auc:0.974189+0.00133614
[504]	train-auc:0.987631+0.000174357	test-auc:0.974204+0.00132999
[505]	train-auc:0.987659+0.000182168	test-auc:0.974206+0.00133746
[506]	train-auc:0.98769+0.000186928	test-auc:0.974208+0.00135363
[507]	train-auc:0.987716+0.000184658	test-auc:0.974212+0.00135668
[508]	train-auc:0.987743+0.000193599	test-auc:0.974216+0.0013601
[509]	train-auc:0.987768+0.000184031	test-auc:0.974213+0.00135945
[510]	train-auc:0.987787+0.000185604	test-auc:0.974221+0.00136487
[511]	train-auc:0.987806+0.00019292	test-auc:0.974234+0.00134152
[512]	train-auc:0.987828+0.000187139	test-auc:0.97425+0.00134357
[513]	train-auc:0.987846+0.000180394	test-auc:0.974252+0.00135242
[514]	train-auc:0.987875+0.000174757	test-auc:0.974266+0.0013536
[515]	train-auc:0.987892+0.000180485	test-auc:0.974267+0.00135102
[516]	train-auc:0.987915+0.000174006	test-auc:0.974267+0.00134696
[517]	train-auc

[627]	train-auc:0.989856+0.000141603	test-auc:0.974757+0.00134225
[628]	train-auc:0.989874+0.0001423	test-auc:0.974767+0.0013382
[629]	train-auc:0.989895+0.000146975	test-auc:0.974761+0.00133645
[630]	train-auc:0.989903+0.000146079	test-auc:0.974768+0.00134113
[631]	train-auc:0.98992+0.000148252	test-auc:0.97477+0.00133836
[632]	train-auc:0.989932+0.00014345	test-auc:0.974768+0.00134252
[633]	train-auc:0.989946+0.000140623	test-auc:0.974772+0.00133739
[634]	train-auc:0.989962+0.000142033	test-auc:0.974773+0.00134628
[635]	train-auc:0.989974+0.000134564	test-auc:0.974771+0.0013473
[636]	train-auc:0.989993+0.000140818	test-auc:0.974788+0.00135232
[637]	train-auc:0.99001+0.000144257	test-auc:0.974788+0.00135022
[638]	train-auc:0.990025+0.000146332	test-auc:0.974796+0.00133901
[639]	train-auc:0.990042+0.000147154	test-auc:0.974803+0.00132589
[640]	train-auc:0.990059+0.000147395	test-auc:0.974799+0.00133393
[641]	train-auc:0.990075+0.000142749	test-auc:0.974796+0.00132467
[642]	train-auc:0.

[753]	train-auc:0.991604+0.000102123	test-auc:0.975158+0.00124181
[754]	train-auc:0.991615+0.000101606	test-auc:0.975165+0.00124391
[755]	train-auc:0.991625+0.000105748	test-auc:0.975165+0.00124181
[756]	train-auc:0.99164+0.000100498	test-auc:0.975174+0.00124426
[757]	train-auc:0.991651+0.000104106	test-auc:0.975183+0.00124409
[758]	train-auc:0.991665+0.000106976	test-auc:0.975182+0.0012489
[759]	train-auc:0.991676+0.000109081	test-auc:0.975179+0.00124768
[760]	train-auc:0.991686+0.000110122	test-auc:0.975181+0.00124628
[761]	train-auc:0.991705+0.000109476	test-auc:0.975186+0.0012385
[762]	train-auc:0.991718+0.0001106	test-auc:0.97519+0.00122986
[763]	train-auc:0.991734+0.000111505	test-auc:0.975189+0.0012355
[764]	train-auc:0.991744+0.000111796	test-auc:0.975194+0.0012417
[765]	train-auc:0.991756+0.000107228	test-auc:0.975191+0.0012371
[766]	train-auc:0.991765+0.000105757	test-auc:0.975199+0.00123501
[767]	train-auc:0.991774+0.000109761	test-auc:0.975194+0.00123984
[768]	train-auc:0.9

[878]	train-auc:0.992966+0.000123315	test-auc:0.975446+0.0010793
[879]	train-auc:0.992973+0.00012384	test-auc:0.975452+0.00108088
[880]	train-auc:0.992981+0.000124359	test-auc:0.975451+0.00108091
[881]	train-auc:0.99299+0.000124035	test-auc:0.975447+0.00107955
[882]	train-auc:0.993003+0.000125462	test-auc:0.975448+0.00107588
[883]	train-auc:0.993011+0.000123644	test-auc:0.975454+0.0010807
[884]	train-auc:0.993019+0.000122946	test-auc:0.975454+0.00108458
[885]	train-auc:0.993029+0.000120697	test-auc:0.975455+0.00108869
[886]	train-auc:0.99304+0.000122071	test-auc:0.975461+0.00108832
[887]	train-auc:0.993047+0.000118655	test-auc:0.975462+0.00107959
[888]	train-auc:0.993058+0.000120392	test-auc:0.975458+0.00108387
[889]	train-auc:0.993064+0.000118424	test-auc:0.975457+0.00108199
[890]	train-auc:0.993076+0.00011497	test-auc:0.975459+0.00108783
[891]	train-auc:0.993087+0.000112964	test-auc:0.975469+0.00109948
[892]	train-auc:0.993093+0.000112294	test-auc:0.97547+0.00110021
[893]	train-auc:0

[1003]	train-auc:0.994108+0.000112597	test-auc:0.975649+0.00111944
[1004]	train-auc:0.994113+0.000112785	test-auc:0.975655+0.001117
[1005]	train-auc:0.99412+0.000113778	test-auc:0.975656+0.0011198
[1006]	train-auc:0.994133+0.000111123	test-auc:0.975661+0.00111047
[1007]	train-auc:0.994142+0.000113217	test-auc:0.975669+0.00110891
[1008]	train-auc:0.994153+0.000117237	test-auc:0.975671+0.00110846
[1009]	train-auc:0.99416+0.000119101	test-auc:0.975668+0.00111785
[1010]	train-auc:0.994169+0.00011683	test-auc:0.975673+0.00111278
[1011]	train-auc:0.994177+0.000114743	test-auc:0.975688+0.00110618
[1012]	train-auc:0.994183+0.000117107	test-auc:0.975692+0.0011058
[1013]	train-auc:0.994192+0.000116651	test-auc:0.975694+0.00110881
[1014]	train-auc:0.994198+0.000116435	test-auc:0.975695+0.00111367
[1015]	train-auc:0.994204+0.000116964	test-auc:0.975691+0.00110848
[1016]	train-auc:0.994212+0.00011723	test-auc:0.975688+0.00110882
[1017]	train-auc:0.994219+0.000117077	test-auc:0.975698+0.00110389
[10

[1127]	train-auc:0.995027+0.00012039	test-auc:0.975739+0.0010263
[1128]	train-auc:0.995032+0.000120725	test-auc:0.975741+0.00102556
[1129]	train-auc:0.995037+0.000120641	test-auc:0.975745+0.00102325
[1130]	train-auc:0.995045+0.000115155	test-auc:0.975745+0.00102044
[1131]	train-auc:0.995051+0.000115316	test-auc:0.97575+0.00102104
[1132]	train-auc:0.995064+0.000111969	test-auc:0.975749+0.00102648
[1133]	train-auc:0.995071+0.000111484	test-auc:0.97575+0.00102854
[1134]	train-auc:0.995077+0.000112564	test-auc:0.975749+0.0010319
[1135]	train-auc:0.995083+0.000113456	test-auc:0.975749+0.00102772
[1136]	train-auc:0.995091+0.000110928	test-auc:0.975747+0.00103422
[1137]	train-auc:0.995099+0.000114101	test-auc:0.975752+0.0010331
[1138]	train-auc:0.995103+0.00011269	test-auc:0.97575+0.00103497
[1139]	train-auc:0.995109+0.000111448	test-auc:0.975751+0.00103817
[1140]	train-auc:0.995114+0.000110409	test-auc:0.975755+0.00103642
[1141]	train-auc:0.995122+0.0001085	test-auc:0.975748+0.00103785
[1142

In [92]:
print('Best number of trees = {}'.format(cvresult.shape[0]))
clf.set_params(n_estimators=cvresult.shape[0])

Best number of trees = 1175


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0.5, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1175, nthread=-1,
       objective='binary:logistic', reg_alpha=0.05, reg_lambda=0.05,
       scale_pos_weight=1, seed=1301, silent=True, subsample=0.7)

In [93]:
print('Fit on the trainingsdata')
clf.fit(x_train, y_train, eval_metric='auc')

Fit on the trainingsdata


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0.5, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1175, nthread=-1,
       objective='binary:logistic', reg_alpha=0.05, reg_lambda=0.05,
       scale_pos_weight=1, seed=1301, silent=True, subsample=0.7)

In [94]:
print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(x_train)[:,1]))

('Overall AUC:', 0.99388631423665053)


In [44]:
#preds = np.zeros((test.shape[0], len(test.columns)))

In [45]:
# preds[:,1] = clf.predict(x_test)

In [95]:
pred = clf.predict_proba(x_test,ntree_limit=cvresult.shape[0])

In [96]:
#submission = pd.DataFrame()
#submission['id'] = test['id']
submission['toxic'] = pred[:,1]
#submission.to_csv("submission.csv", index=False)

In [48]:
# lin_clf = svm.LinearSVC(C = 0.25)

In [49]:
# lin_clf

In [50]:
# lin_clf.fit(x_train,y_train)

In [51]:
# print('Overall AUC:', roc_auc_score(y_train, lin_clf.decision_function(x_train)))

# SEVERE_TOXIC#

In [97]:
y_train = train['severe_toxic']
x_train = x_train

In [98]:
xgtrain = xgb.DMatrix(x_train, y_train)

In [99]:
clf = xgb.XGBClassifier(max_depth = 3,
                n_estimators=5000,
                learning_rate=0.1, 
                gamma = 0.5,
                reg_lambda = 0.05,
                reg_alpha = 0.05,
                #nthread=4,
                subsample=0.7,
                colsample_bytree=0.7,
                min_child_weight = 1,
                seed=1301)
xgb_param = clf.get_xgb_params()

In [100]:
#do cross validation
print ('Start cross validation')
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=5000, nfold=5, metrics=['auc'],
     early_stopping_rounds=50, stratified=True, seed=1301,verbose_eval = 1)

Start cross validation
[0]	train-auc:0.865316+0.0403896	test-auc:0.852963+0.035234
[1]	train-auc:0.898161+0.0131003	test-auc:0.892439+0.0141151
[2]	train-auc:0.91089+0.0157295	test-auc:0.908194+0.00977664
[3]	train-auc:0.92414+0.00651548	test-auc:0.916473+0.00301395
[4]	train-auc:0.926703+0.00660906	test-auc:0.91807+0.00407241
[5]	train-auc:0.932614+0.00506651	test-auc:0.920239+0.00370153
[6]	train-auc:0.933716+0.005444	test-auc:0.921359+0.00583241
[7]	train-auc:0.937262+0.00794068	test-auc:0.923775+0.009616
[8]	train-auc:0.941233+0.0100011	test-auc:0.931679+0.0106512
[9]	train-auc:0.943794+0.0117959	test-auc:0.933228+0.0120558
[10]	train-auc:0.946522+0.00951284	test-auc:0.938087+0.00835181
[11]	train-auc:0.947946+0.0092242	test-auc:0.940174+0.00560335
[12]	train-auc:0.953086+0.00926105	test-auc:0.944774+0.00242502
[13]	train-auc:0.956022+0.00845375	test-auc:0.950415+0.00508656
[14]	train-auc:0.959142+0.00761749	test-auc:0.953636+0.00645881
[15]	train-auc:0.961177+0.00882802	test-auc:0

[127]	train-auc:0.994735+0.000127943	test-auc:0.987242+0.00127605
[128]	train-auc:0.994769+0.000129643	test-auc:0.987253+0.00129511
[129]	train-auc:0.994817+0.00012062	test-auc:0.987244+0.00130101
[130]	train-auc:0.994858+0.000111275	test-auc:0.987264+0.00129259
[131]	train-auc:0.994899+0.00010954	test-auc:0.987269+0.00127596
[132]	train-auc:0.994927+0.000103163	test-auc:0.987261+0.00128412
[133]	train-auc:0.994956+0.000110641	test-auc:0.987275+0.00129659
[134]	train-auc:0.994995+0.000113231	test-auc:0.987306+0.00125169
[135]	train-auc:0.995034+0.000107612	test-auc:0.987337+0.0012406
[136]	train-auc:0.995073+0.000110143	test-auc:0.987353+0.00124133
[137]	train-auc:0.995109+9.83004e-05	test-auc:0.98733+0.00124963
[138]	train-auc:0.995138+9.14562e-05	test-auc:0.987292+0.0012648
[139]	train-auc:0.995167+9.43474e-05	test-auc:0.987297+0.00124069
[140]	train-auc:0.9952+0.000100007	test-auc:0.987303+0.00125889
[141]	train-auc:0.995239+0.000106961	test-auc:0.987331+0.00128123
[142]	train-auc:0

In [101]:
print('Best number of trees = {}'.format(cvresult.shape[0]))

Best number of trees = 181


In [102]:
clf.set_params(n_estimators=cvresult.shape[0])

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0.5, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=181, nthread=-1,
       objective='binary:logistic', reg_alpha=0.05, reg_lambda=0.05,
       scale_pos_weight=1, seed=1301, silent=True, subsample=0.7)

In [103]:
print('Fit on the trainingsdata')
clf.fit(x_train, y_train, eval_metric='auc')

Fit on the trainingsdata


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0.5, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=181, nthread=-1,
       objective='binary:logistic', reg_alpha=0.05, reg_lambda=0.05,
       scale_pos_weight=1, seed=1301, silent=True, subsample=0.7)

In [104]:
print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(x_train)[:,1]))

('Overall AUC:', 0.99554362092698334)


In [105]:
pred = clf.predict_proba(x_test,ntree_limit=cvresult.shape[0])

In [106]:
submission['severe_toxic'] = pred[:,1]
#submission.to_csv("submission.csv", index=False)

# OBSCENE#

In [107]:
y_train = train['obscene']
x_train = x_train

In [108]:
xgtrain = xgb.DMatrix(x_train, y_train)

In [ ]:
clf = xgb.XGBClassifier(max_depth = 3,
                n_estimators=5000,
                learning_rate=0.1, 
                gamma = 0.5,
                reg_lambda = 0.05,
                reg_alpha = 0.05,
                #nthread=4,
                subsample=0.7,
                colsample_bytree=0.7,
                min_child_weight = 1,
                seed=1301)
xgb_param = clf.get_xgb_params()

In [ ]:
#do cross validation
print ('Start cross validation')
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=5000, nfold=5, metrics=['auc'],
     early_stopping_rounds=60, stratified=True, seed=1301,verbose_eval = 1)

Start cross validation
[0]	train-auc:0.816366+0.0111931	test-auc:0.813142+0.0138449
[1]	train-auc:0.862662+0.0207908	test-auc:0.860788+0.0177831
[2]	train-auc:0.875202+0.00967985	test-auc:0.874272+0.00963699
[3]	train-auc:0.885205+0.00539991	test-auc:0.88381+0.00923868
[4]	train-auc:0.890588+0.00835829	test-auc:0.889859+0.0108326
[5]	train-auc:0.896706+0.00913746	test-auc:0.896724+0.0123412
[6]	train-auc:0.901325+0.00709043	test-auc:0.90038+0.0119516
[7]	train-auc:0.911039+0.00547341	test-auc:0.909947+0.00847895
[8]	train-auc:0.916174+0.00541129	test-auc:0.913736+0.00861963
[9]	train-auc:0.920006+0.00420392	test-auc:0.917916+0.00750881
[10]	train-auc:0.926441+0.00231456	test-auc:0.92446+0.00487336
[11]	train-auc:0.932028+0.00350161	test-auc:0.92996+0.00423512
[12]	train-auc:0.938091+0.0028689	test-auc:0.935737+0.0054139
[13]	train-auc:0.94053+0.00286417	test-auc:0.937576+0.00436093
[14]	train-auc:0.942645+0.00182662	test-auc:0.939955+0.003247
[15]	train-auc:0.945574+0.00125766	test-auc

[126]	train-auc:0.990113+0.000250829	test-auc:0.985428+0.000790752
[127]	train-auc:0.990167+0.00028026	test-auc:0.985476+0.000755891
[128]	train-auc:0.990212+0.000277636	test-auc:0.98551+0.000780261
[129]	train-auc:0.990245+0.000279592	test-auc:0.985532+0.000778089
[130]	train-auc:0.990322+0.000260945	test-auc:0.985625+0.000799976
[131]	train-auc:0.990385+0.000292839	test-auc:0.985693+0.000810316
[132]	train-auc:0.990448+0.000308948	test-auc:0.98574+0.000792181
[133]	train-auc:0.990507+0.000313088	test-auc:0.985764+0.000788266
[134]	train-auc:0.990547+0.000287665	test-auc:0.985787+0.000772666
[135]	train-auc:0.9906+0.000285788	test-auc:0.985824+0.000771973
[136]	train-auc:0.990638+0.000273292	test-auc:0.985857+0.000789416
[137]	train-auc:0.990672+0.000285708	test-auc:0.98589+0.000767496
[138]	train-auc:0.990711+0.000281243	test-auc:0.985909+0.000789539
[139]	train-auc:0.990772+0.000286902	test-auc:0.985947+0.000745689
[140]	train-auc:0.99082+0.000268989	test-auc:0.985972+0.000751632
[1

[250]	train-auc:0.994036+0.000208473	test-auc:0.988015+0.000765714
[251]	train-auc:0.994054+0.000212775	test-auc:0.988029+0.000762303
[252]	train-auc:0.994069+0.000217455	test-auc:0.988042+0.000754579
[253]	train-auc:0.994097+0.000220024	test-auc:0.988047+0.000755687
[254]	train-auc:0.994119+0.000213187	test-auc:0.988067+0.000766638
[255]	train-auc:0.994133+0.000209495	test-auc:0.988075+0.000774193
[256]	train-auc:0.994152+0.000211089	test-auc:0.988084+0.000743215
[257]	train-auc:0.99417+0.000207549	test-auc:0.988108+0.000769518
[258]	train-auc:0.994181+0.000210177	test-auc:0.988116+0.000773438
[259]	train-auc:0.994201+0.000217457	test-auc:0.988121+0.000772177
[260]	train-auc:0.99422+0.000214619	test-auc:0.988128+0.000767201
[261]	train-auc:0.994236+0.000209715	test-auc:0.988144+0.000757653
[262]	train-auc:0.994265+0.000208334	test-auc:0.988159+0.000763344
[263]	train-auc:0.994282+0.000203586	test-auc:0.988175+0.000766711
[264]	train-auc:0.994296+0.000206358	test-auc:0.988182+0.0007647

In [ ]:
print('Best number of trees = {}'.format(cvresult.shape[0]))
clf.set_params(n_estimators=cvresult.shape[0])

In [ ]:
print('Fit on the trainingsdata')
clf.fit(x_train, y_train, eval_metric='auc')

In [ ]:
print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(x_train)[:,1]))

In [ ]:
pred = clf.predict_proba(x_test, ntree_limit=cvresult.shape[0])

In [ ]:
submission['obscene'] = pred[:,1]
#submission.to_csv("submission.csv", index=False)

# THREAT#

In [ ]:
y_train = train['threat']
x_train = x_train

In [ ]:
xgtrain = xgb.DMatrix(x_train, y_train)

In [ ]:
clf = xgb.XGBClassifier(max_depth = 3,
                n_estimators=5000,
                learning_rate=0.1, 
                gamma = 0.5,
                reg_lambda = 0.05,
                reg_alpha = 0.05,
                #nthread=4,
                subsample=0.7,
                colsample_bytree=0.7,
                min_child_weight = 1,
                seed=1301)
xgb_param = clf.get_xgb_params()

In [ ]:
#do cross validation
print ('Start cross validation')
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=5000, nfold=5, metrics=['auc'],
     early_stopping_rounds=60, stratified=True, seed=1301,verbose_eval = 1)

In [ ]:
print('Best number of trees = {}'.format(cvresult.shape[0]))
clf.set_params(n_estimators=cvresult.shape[0])

In [ ]:
print('Fit on the trainingsdata')
clf.fit(x_train, y_train, eval_metric='auc')

In [ ]:
print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(x_train)[:,1]))

In [ ]:
pred = clf.predict_proba(x_test, ntree_limit=cvresult.shape[0])

In [ ]:
submission['threat'] = pred[:,1]
#submission.to_csv("submission.csv", index=False)

 # INSULT#

In [ ]:
y_train = train['insult']
x_train = x_train

In [ ]:
xgtrain = xgb.DMatrix(x_train, y_train)

In [ ]:
clf = xgb.XGBClassifier(max_depth = 3,
                n_estimators=5000,
                learning_rate=0.1, 
                gamma = 0.5,
                reg_lambda = 0.05,
                reg_alpha = 0.05,
                #nthread=4,
                subsample=0.7,
                colsample_bytree=0.7,
                min_child_weight = 1,
                seed=1301)
xgb_param = clf.get_xgb_params()

In [ ]:
#do cross validation
print ('Start cross validation')
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=5000, nfold=5, metrics=['auc'],
     early_stopping_rounds=60, stratified=True, seed=1301,verbose_eval = 1)

In [ ]:
print('Best number of trees = {}'.format(cvresult.shape[0]))
clf.set_params(n_estimators=cvresult.shape[0])

In [ ]:
print('Fit on the training data')
clf.fit(x_train, y_train, eval_metric='auc')

In [ ]:
print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(x_train)[:,1]))

In [ ]:
pred = clf.predict_proba(x_test, ntree_limit=cvresult.shape[0])

In [ ]:
submission['insult'] = pred[:,1]
#submission.to_csv("submission.csv", index=False)

In [ ]:
submission.head(1)

# IDENTITY_HATE#

In [24]:
y_train = train['identity_hate']
#x_train = xtrain_glove
x_train = x_train

In [25]:
xgtrain = xgb.DMatrix(x_train, y_train)

In [34]:
clf = xgb.XGBClassifier(max_depth = 3,
                n_estimators=5000,
                learning_rate=0.05, 
                gamma = 0.5,
                reg_lambda = 0.05,
                reg_alpha = 0.05,
                #nthread=4,
                subsample=0.8,
                colsample_bytree=0.7,
                min_child_weight = 1,
                seed=1301)
xgb_param = clf.get_xgb_params()

In [35]:
#do cross validation
print ('Start cross validation')
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=5000, nfold=5, metrics=['auc'],
early_stopping_rounds=110, 
                  stratified=True, seed=1301,verbose_eval = 1)

Start cross validation
[0]	train-auc:0.734684+0.068095	test-auc:0.726889+0.0645371
[1]	train-auc:0.794533+0.0806031	test-auc:0.785202+0.0796098
[2]	train-auc:0.842262+0.0389906	test-auc:0.829205+0.0360194
[3]	train-auc:0.853364+0.0421633	test-auc:0.84547+0.0353397
[4]	train-auc:0.85336+0.0421459	test-auc:0.845516+0.0353899
[5]	train-auc:0.865573+0.0473196	test-auc:0.860323+0.042192
[6]	train-auc:0.868966+0.043892	test-auc:0.862348+0.0388795
[7]	train-auc:0.881041+0.0258112	test-auc:0.875972+0.0210178
[8]	train-auc:0.884462+0.0239795	test-auc:0.880029+0.0198079
[9]	train-auc:0.895506+0.0165528	test-auc:0.890069+0.013594
[10]	train-auc:0.896831+0.0169305	test-auc:0.891604+0.0158353
[11]	train-auc:0.901117+0.0179897	test-auc:0.894946+0.0152833
[12]	train-auc:0.906049+0.0133792	test-auc:0.900416+0.00959126
[13]	train-auc:0.913434+0.00772568	test-auc:0.906258+0.0115623
[14]	train-auc:0.916951+0.00694558	test-auc:0.909577+0.00939086
[15]	train-auc:0.918955+0.0071768	test-auc:0.913316+0.00675

[129]	train-auc:0.980434+0.00106247	test-auc:0.970251+0.00519687
[130]	train-auc:0.980578+0.00106847	test-auc:0.970311+0.00526255
[131]	train-auc:0.980728+0.00105184	test-auc:0.970375+0.00526047
[132]	train-auc:0.980901+0.00100755	test-auc:0.97047+0.0052437
[133]	train-auc:0.981128+0.00099021	test-auc:0.970565+0.00519671
[134]	train-auc:0.981254+0.000984765	test-auc:0.970599+0.00518014
[135]	train-auc:0.98138+0.000971796	test-auc:0.970576+0.00530062
[136]	train-auc:0.981534+0.000969062	test-auc:0.970662+0.00534587
[137]	train-auc:0.981642+0.000971502	test-auc:0.970857+0.00528156
[138]	train-auc:0.981754+0.000974673	test-auc:0.970892+0.00525334
[139]	train-auc:0.981872+0.000984837	test-auc:0.970925+0.00523061
[140]	train-auc:0.982017+0.000998326	test-auc:0.970926+0.00524799
[141]	train-auc:0.982128+0.000987999	test-auc:0.971135+0.00518714
[142]	train-auc:0.982244+0.000987232	test-auc:0.971207+0.00510625
[143]	train-auc:0.982372+0.000952595	test-auc:0.971315+0.00505602
[144]	train-auc:0.

[255]	train-auc:0.990604+0.000565375	test-auc:0.975721+0.00501697
[256]	train-auc:0.990654+0.000561531	test-auc:0.975722+0.00504494
[257]	train-auc:0.990713+0.000551465	test-auc:0.975753+0.00502402
[258]	train-auc:0.990745+0.000549882	test-auc:0.975786+0.00500292
[259]	train-auc:0.990786+0.000542963	test-auc:0.975795+0.00498982
[260]	train-auc:0.99082+0.00054059	test-auc:0.975808+0.00494831
[261]	train-auc:0.990852+0.000532862	test-auc:0.975825+0.00497662
[262]	train-auc:0.990894+0.000528276	test-auc:0.975823+0.00500306
[263]	train-auc:0.990924+0.000541558	test-auc:0.975838+0.00504198
[264]	train-auc:0.990965+0.000528009	test-auc:0.975876+0.00508644
[265]	train-auc:0.991004+0.000527009	test-auc:0.975893+0.00507022
[266]	train-auc:0.991047+0.000506074	test-auc:0.975918+0.00507625
[267]	train-auc:0.991097+0.000517279	test-auc:0.975947+0.00509005
[268]	train-auc:0.991144+0.000527548	test-auc:0.975934+0.00509394
[269]	train-auc:0.991177+0.000539912	test-auc:0.975958+0.00506225
[270]	train-

[381]	train-auc:0.994684+0.000357821	test-auc:0.977082+0.00510698
[382]	train-auc:0.994711+0.000352768	test-auc:0.977099+0.00510916
[383]	train-auc:0.994734+0.000354522	test-auc:0.977117+0.00510124
[384]	train-auc:0.994758+0.000353091	test-auc:0.977108+0.0051168
[385]	train-auc:0.994784+0.000350381	test-auc:0.977117+0.00511443
[386]	train-auc:0.994816+0.000351731	test-auc:0.977127+0.00512345
[387]	train-auc:0.99484+0.000353998	test-auc:0.977136+0.00513783
[388]	train-auc:0.99486+0.000355766	test-auc:0.977124+0.00515104
[389]	train-auc:0.994885+0.000355127	test-auc:0.977112+0.00518284
[390]	train-auc:0.994909+0.000350581	test-auc:0.977126+0.00519409
[391]	train-auc:0.99493+0.000346378	test-auc:0.977101+0.0052272
[392]	train-auc:0.994961+0.000349461	test-auc:0.977113+0.00524559
[393]	train-auc:0.994983+0.000348577	test-auc:0.977136+0.00520454
[394]	train-auc:0.995003+0.000348979	test-auc:0.977143+0.00520122
[395]	train-auc:0.995019+0.000353273	test-auc:0.977126+0.00520787
[396]	train-auc

[507]	train-auc:0.996898+0.000227124	test-auc:0.977665+0.00535454
[508]	train-auc:0.99691+0.000230515	test-auc:0.977664+0.00538671
[509]	train-auc:0.99692+0.000231119	test-auc:0.977634+0.00539626
[510]	train-auc:0.996937+0.000227321	test-auc:0.977663+0.0053602
[511]	train-auc:0.996957+0.000225431	test-auc:0.977641+0.00539838
[512]	train-auc:0.996971+0.000216112	test-auc:0.977653+0.00540756
[513]	train-auc:0.996981+0.000219561	test-auc:0.977656+0.00538667
[514]	train-auc:0.996996+0.000219329	test-auc:0.977622+0.00542767
[515]	train-auc:0.997008+0.000218435	test-auc:0.977651+0.0054265
[516]	train-auc:0.997016+0.000216926	test-auc:0.977637+0.00546853
[517]	train-auc:0.997031+0.000210227	test-auc:0.977629+0.00549245
[518]	train-auc:0.99704+0.000206466	test-auc:0.977641+0.00546522
[519]	train-auc:0.997051+0.000209115	test-auc:0.977621+0.00549816
[520]	train-auc:0.997063+0.000208909	test-auc:0.977623+0.00552201
[521]	train-auc:0.997079+0.000211284	test-auc:0.977658+0.00549817
[522]	train-auc

[633]	train-auc:0.998185+0.000123572	test-auc:0.978102+0.00544659
[634]	train-auc:0.998192+0.000122568	test-auc:0.978113+0.00544573
[635]	train-auc:0.998202+0.000127305	test-auc:0.978133+0.00547116
[636]	train-auc:0.998213+0.000133471	test-auc:0.978118+0.00549981
[637]	train-auc:0.998221+0.00013418	test-auc:0.978121+0.00551511
[638]	train-auc:0.998228+0.000130386	test-auc:0.978156+0.00549503
[639]	train-auc:0.998235+0.000129571	test-auc:0.978156+0.00550938
[640]	train-auc:0.99824+0.000129798	test-auc:0.978149+0.00550545
[641]	train-auc:0.998246+0.000128314	test-auc:0.978145+0.00553484
[642]	train-auc:0.998255+0.000132987	test-auc:0.978134+0.00555384
[643]	train-auc:0.998258+0.000133182	test-auc:0.978125+0.00555532
[644]	train-auc:0.998263+0.000132628	test-auc:0.978124+0.00555817
[645]	train-auc:0.998274+0.000132229	test-auc:0.97813+0.00555235
[646]	train-auc:0.998278+0.000132678	test-auc:0.978132+0.00554779
[647]	train-auc:0.998282+0.00012958	test-auc:0.978137+0.00554874
[648]	train-au

[759]	train-auc:0.998946+0.000100543	test-auc:0.978263+0.0059715
[760]	train-auc:0.998951+0.000100805	test-auc:0.978253+0.00599876
[761]	train-auc:0.998955+0.000100279	test-auc:0.978261+0.00601064
[762]	train-auc:0.998958+0.000100089	test-auc:0.978263+0.00601123
[763]	train-auc:0.998961+9.95783e-05	test-auc:0.978262+0.00602073
[764]	train-auc:0.998968+0.000102167	test-auc:0.978252+0.0060424
[765]	train-auc:0.998973+0.000102889	test-auc:0.978255+0.00603645
[766]	train-auc:0.998975+0.000102517	test-auc:0.978238+0.00606213
[767]	train-auc:0.998977+0.000103314	test-auc:0.978234+0.00606986
[768]	train-auc:0.998983+0.000105714	test-auc:0.978242+0.00607898
[769]	train-auc:0.998986+0.000102779	test-auc:0.978262+0.00608697
[770]	train-auc:0.998991+0.000102655	test-auc:0.978253+0.00610322
[771]	train-auc:0.998997+0.000101802	test-auc:0.978248+0.00610487
[772]	train-auc:0.999001+9.98511e-05	test-auc:0.978241+0.00611574
[773]	train-auc:0.999005+9.89727e-05	test-auc:0.978252+0.00611229
[774]	train-

[884]	train-auc:0.999408+7.11916e-05	test-auc:0.978264+0.00610884
[885]	train-auc:0.999413+7.12893e-05	test-auc:0.97826+0.00612254
[886]	train-auc:0.999415+7.12051e-05	test-auc:0.978259+0.00612436
[887]	train-auc:0.999418+6.95212e-05	test-auc:0.978256+0.0061261
[888]	train-auc:0.999421+7.04574e-05	test-auc:0.978238+0.00614053
[889]	train-auc:0.999425+6.97699e-05	test-auc:0.978251+0.0061438
[890]	train-auc:0.999429+7.13667e-05	test-auc:0.978265+0.00612257
[891]	train-auc:0.999433+7.01866e-05	test-auc:0.978262+0.00611866
[892]	train-auc:0.999435+6.84327e-05	test-auc:0.978289+0.0061317
[893]	train-auc:0.999439+6.83737e-05	test-auc:0.978284+0.00612807
[894]	train-auc:0.999442+6.90606e-05	test-auc:0.978273+0.0061314
[895]	train-auc:0.999444+6.79164e-05	test-auc:0.97828+0.0061278
[896]	train-auc:0.999446+6.80694e-05	test-auc:0.978283+0.00611496
[897]	train-auc:0.999448+6.91103e-05	test-auc:0.978298+0.00610824
[898]	train-auc:0.999449+6.81762e-05	test-auc:0.978296+0.00611241
[899]	train-auc:0

In [36]:
print('Best number of trees = {}'.format(cvresult.shape[0]))
clf.set_params(n_estimators=cvresult.shape[0])

Best number of trees = 794


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0.5, learning_rate=0.05, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=794, nthread=-1,
       objective='binary:logistic', reg_alpha=0.05, reg_lambda=0.05,
       scale_pos_weight=1, seed=1301, silent=True, subsample=0.8)

In [37]:
print('Fit on the trainingsdata')
clf.fit(x_train, y_train, eval_metric='auc')

Fit on the trainingsdata


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0.5, learning_rate=0.05, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=794, nthread=-1,
       objective='binary:logistic', reg_alpha=0.05, reg_lambda=0.05,
       scale_pos_weight=1, seed=1301, silent=True, subsample=0.8)

In [38]:
print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(x_train)[:,1]))

('Overall AUC:', 0.99838719156408628)


In [39]:
x_test.shape

(153164, 50307)

In [40]:
x_train.shape

(159571, 50307)

In [41]:
pred = clf.predict_proba(x_test, ntree_limit=cvresult.shape[0])

In [42]:
submission['identity_hate'] = pred[:,1]
#submission.to_csv("submission.csv", index=False)

In [146]:
#submission = pd.read_csv("C:\Users\Admin\Desktop\submission.csv")
submission = submission.reset_index(drop = True)

In [ ]:
#Rearrange Columns

In [147]:
submission.head(3)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.998805,0.184579,0.996780,0.131857,0.975292,0.352496
1,0000247867823ef7,0.010012,0.000048,0.002724,0.000051,0.004865,0.000430
2,00013b17ad220c46,0.011465,0.000411,0.003770,0.000165,0.001175,0.000680


In [194]:
submission.to_csv("submission.csv", index=False)

In [118]:
#Let's calculate the local; CV
Local_CV = (0.975108+0.980057+0.981987+0.989207+0.987836+0.975613)/6
Local_CV

0.9816346666666668

In [148]:
#Score on LB.Not Submitting native.Blending it with one DL
0.9766

0.9766

In [150]:
# #Local CV - Kaggle LB.
0.9816346666666668 - 0.9766

0.005034666666666743

In [ ]:
#Let's target bringing each model upto .9870